# Notebook description

Decoders $d$ are found via optimization to minimize the mean-squared error between then desired function and the decode-weighted tuning curve as well as the L2 and L1 norms of the decode weights themselves.

$$\arg\min_d \|f-Ad\|_2^2 + \lambda_{L2}\|d\|_2 + \lambda_{L1}\|d\|_1$$

 - Generate realistic tuning curves from hardware
 - Specify an exemplary target function to decode (e.g. $f(x)=f_{max}(x+1)$)
 - Sweep the space of $\lambda_{L2}$ and $\lambda_{L1}$, finding decoder sets for each $\lambda_{L1}$ $\lambda_{L2}$ pair
 - Examine the decode validation error, input frequency (i.e., energy), nonzero decode weights, and SNR at 0 input for each $\lambda_{L1}$ $\lambda_{L2}$ pair

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from nengo_brainstorm.solvers import CVXSolver

import pystorm
from pystorm.hal import HAL
from pystorm.hal.net_builder import NetBuilder
from pystorm.hal.run_control import RunControl
from pystorm.hal.data_utils import lpf, bin_to_spk_times, bins_to_rates

In [ ]:
# set parameters for network
X = 8
Y = 8
NNEURON = X*Y
DIM = 1
FMAX = 1000
DOWNSTREAM_NS = 10000
UPSTREAM_NS   = 1000000

In [ ]:
hal = pystorm.hal.HAL()
net_builder = NetBuilder(hal)

def build_taps(net_builder):
    bad_syn, _ = net_builder.determine_bad_syns()
    SX = X // 2
    SY = Y // 2
    bad_syn = bad_syn[:SY, :SX]
    tap_matrix_syn = net_builder.create_default_yx_taps(SY, SX, DIM, bad_syn)
    tap_matrix = net_builder.syn_taps_to_nrn_taps(tap_matrix_syn)
    np.savetxt("tap_matrix.txt", tap_matrix)
    return tap_matrix

tap_matrix = build_taps(net_builder)


In [ ]:
def build_net(net_builder, tap_matrix, d_matrix):
    gain_divs = np.loadtxt("gain_divisors.txt", dtype=int)
    biases = np.loadtxt("biases.txt", dtype=int)
    d_matrix = np.eye(Y*X)
    net = net_builder.create_single_pool_net(
        Y, X, tap_matrix, biases=biases, gain_divs=gain_divs, decoders=d_matrix)
    return net

tuning_net = build_net(net_builder, tap_matrix, np.eye(Y*X))
run_control = RunControl(hal, tuning_net)
hal.map(tuning_net)

In [ ]:
def collect_tuning_data(net, hal, fmax, run_control, total_points):
    bin_size = 0.5 # seconds
    bin_size_ns = int(bin_size*1E9)
    hal.set_time_resolution(DOWNSTREAM_NS, UPSTREAM_NS)
    input_rates = np.zeros((total_points+1, 1))
    input_rates[:total_points, 0] = fmax * np.linspace(-1, 1, total_points)
    input_rates[-1, 0] = input_rates[-2, 0]
    time_ns = np.arange(total_points+1)*bin_size_ns
    input_data = {net.input:(time_ns, input_rates)}
    output_data, _ = run_control.run_input_sweep(
        input_data, get_raw_spikes=False, get_outputs=True)
    outputs, output_times = output_data
    outputs = outputs[net.output][:, :-1] # last dimension reserved for decode
    spike_rates = bins_to_rates(outputs, output_times, time_ns, init_discard_frac=0.25)
    input_rates = input_rates[:-1]
    return input_rates, spike_rates

def collect_fake_tuning_data(nn, total_points, fmax, max_spike_rate, sigma=20):
    input_values = np.linspace(-1, 1, total_points)
    biases = np.zeros(nn)
    biases[:nn//2] = np.linspace(-1, 1, nn//2)
    biases[nn//2:] = np.linspace(-1, 1, nn//2)
    input_rates = fmax * input_values.reshape((-1, 1))
    spike_rates = np.zeros((total_points, nn))
    spike_rates[:, :nn//2] = max_spike_rate * (input_values.reshape((-1, 1)) + biases[:nn//2])
    spike_rates[:, nn//2:] = max_spike_rate * (-input_values.reshape((-1, 1)) + biases[nn//2:])
    spike_rates += sigma*np.random.randn(*spike_rates.shape)
    spike_rates[spike_rates < 0] = 0
    return input_rates, spike_rates
    
input_rates, spike_rates = collect_tuning_data(tuning_net, hal, FMAX, run_control, total_points=241)
# input_rates, spike_rates = collect_fake_tuning_data(X*Y, total_points=101, fmax=FMAX, max_spike_rate=500)

# split into training and validation data sets
r_valid = 4
v_idx = np.zeros(len(input_rates), dtype=bool)
v_idx[r_valid-r_valid//2::4] = True
train_input_rates = input_rates[~v_idx]
valid_input_rates = input_rates[v_idx]
train_spike_rates = spike_rates[~v_idx]
valid_spike_rates = spike_rates[v_idx]

In [ ]:
def plot_tuning(inputs, spike_rates, array_width, array_height):
    half_width = array_width//2
    plt.figure()
    for idx in range(array_height):
        start_l = idx*array_width
        start_r = start_l + half_width
        plt.plot(inputs, spike_rates[:, start_l:start_l+half_width], 'r')
        plt.plot(inputs, spike_rates[:, start_r:start_r+half_width], 'b')
plot_tuning(train_input_rates, train_spike_rates, X, Y)
plt.title("training")
plt.xlabel("input rate")
plt.ylabel("spike rate")
plot_tuning(valid_input_rates, valid_spike_rates, X, Y)
plt.title("validation")
plt.xlabel("input rate")
plt.ylabel("spike rate");

In [ ]:
def fit_decoders(rates, target_function, l1, l2):
    solver = CVXSolver(reg=l2, reg_l1=l1)
    decoders, info = solver(rates, target_function)
    decoders = decoders.clip(-1, 1)
    return decoders, info

# train_target = train_input_rates + FMAX
# valid_target = valid_input_rates + FMAX

def target_function(rates, freq=0.5):
    return FMAX*(np.sin(2*np.pi*freq*rates/FMAX))
train_target = target_function(train_input_rates)
valid_target = target_function(valid_input_rates)

def plot_target_fcn():
    plt.plot(train_input_rates, train_target, 'o', label="training targets ({})".format(len(train_input_rates)))
    plt.plot(valid_input_rates, valid_target, 'o', label="validation targets ({})".format(len(valid_input_rates)))
    plt.legend(loc="best")
    plt.xlabel("Input Spike Rate")
plot_target_fcn()

In [ ]:
# Set up L1 L2 space to sweep
# N_L1 = 25
# N_L2 = 25
# L1_vals = np.linspace(0.1, 10.0, N_L1)
# L2_vals = np.linspace(0.1, 10.0, N_L2)

N_L1 = 11
N_L2 = 11
L1_vals = np.logspace(-1, 6, N_L1)
L2_vals = np.logspace(-1, 6, N_L2)


L1_grid, L2_grid = np.meshgrid(L1_vals, L2_vals)
L1L2_pts = np.zeros((N_L1*N_L2, 2))
L1L2_pts[:, 0] = L1_grid.flatten()
L1L2_pts[:, 1] = L2_grid.flatten()

In [ ]:
# Compute decoders
decoders = []
rmse_train = []
for l1, l2 in L1L2_pts:
    dec, info = fit_decoders(train_spike_rates, train_target, l1, l2)
    decoders.append(dec)
    rmse_train.append(info['rmses'])

# from multiprocessing import Pool
# def mp_fit_decoders(l1l2):
#     l1, l2 = l1l2
#     return fit_decoders(train_spike_rates, train_target, l1, l2)
# mp_pool = Pool(processes=4)
# ret = mp_pool.map(mp_fit_decoders, L1L2_pts))

In [ ]:
# Compute stats
nz_dw_threshold = 1/(2*64)

rmse_valid = []
f_in = []
nz_dw = []
mean_abs_dw = []
decodes = []
for dweights in decoders:
    nz_idx = np.abs(dweights.flatten())>nz_dw_threshold
    nz_dw.append(np.sum(nz_idx))
    mean_abs_dw.append(np.mean(np.abs(dweights)))
    f_in.append(np.sum(np.mean(valid_spike_rates[:, nz_idx], axis=0)))
    decode = np.dot(valid_spike_rates, dweights)
    decodes.append(decode)
    plt.plot(valid_input_rates, decode)
    rmse_valid.append(np.sqrt(np.mean((valid_target-decode)**2)))
plt.plot(valid_input_rates, valid_target, 'r')

In [ ]:
# Plot results
rmse_train = np.reshape(rmse_train, (N_L2, N_L1))
rmse_valid = np.reshape(rmse_valid, (N_L2, N_L1))
f_in = np.reshape(f_in, (N_L2, N_L1))
nz_dw = np.reshape(nz_dw, (N_L2, N_L1))
mean_abs_dw = np.reshape(mean_abs_dw, (N_L2, N_L1))

In [ ]:
def plot_decodes(valid_input_rates, valid_target, decodes):
    fig, axs = plt.subplots(nrows=N_L2, ncols=N_L1, sharex=True, sharey=True, figsize=(14, 12))
    for d_idx, decode in enumerate(decodes):
        r_idx = N_L2 - (d_idx // N_L1) - 1
        c_idx = d_idx % N_L1
        axs[r_idx][c_idx].plot(valid_input_rates, valid_target)
        axs[r_idx][c_idx].plot(valid_input_rates, decode)
    axs[0, 0].set_xticks([])
    axs[0, 0].set_yticks([])
plot_decodes(valid_input_rates, valid_target, decodes)

In [ ]:
def plot_contour(X, Y, Z, ax, fig, contour_args={}, cb_args={}):
    cset = ax.contourf(X, Y, Z, **contour_args)
    ax.contour(X, Y, Z, cset.levels, colors="k", linewidths=1)
    ax.set_xscale("log")
    ax.set_yscale("log")
    cb = fig.colorbar(cset, ax=ax, **cb_args)
    ax.set_xlabel("L1 Scaling")
    ax.set_ylabel("L2 Scaling")
    
def plot_rmse(L1, L2, train, valid):
    norm = cm.colors.Normalize(vmin=np.min((rmse_train, rmse_valid)), vmax=np.max((rmse_train, rmse_valid)))
    fig, _axs = plt.subplots(ncols=2, figsize=(12, 4))
    ax_train, ax_valid = _axs
    norm = cm.colors.Normalize(vmin=np.min((train, valid)), vmax=np.max((train, valid)))
    plot_contour(L1, L2, train, ax_train, fig, dict(norm=norm))
    plot_contour(L1, L2, valid, ax_valid, fig, dict(norm=norm))
    ax_train.set_title("Training NRMSE")
    ax_valid.set_title("Validation NRMSE")
plot_rmse(L1_grid, L2_grid, rmse_train/FMAX, rmse_valid/FMAX)

def plot_energy(L1, L2, f_in, nz_dw, mean_abs_dw):
    fig, _axs = plt.subplots(ncols=3, figsize=(16, 4))
    ax_fin, ax_nzdw, ax_madw = _axs
    plot_contour(L1, L2, f_in, ax_fin, fig)
    plot_contour(L1, L2, nz_dw, ax_nzdw, fig)
    plot_contour(L1, L2, mean_abs_dw, ax_madw, fig)
    ax_fin.set_title("f_in")
    ax_nzdw.set_title("Nonzero Decode Weights")
    ax_madw.set_title("Mean Absolute Decode Weight")
plot_energy(L1_grid, L2_grid, f_in, nz_dw, mean_abs_dw)

In [ ]:
# Collect SNR data


In [ ]:
# def plot_training_fit(train_input_rates, target_function, spike_rates, decoders):
#     train_decode = np.dot(spike_rates, decoders)
#     plt.figure()
#     plt.plot(train_input_rates, target_function, label="target function")
#     plt.plot(train_input_rates, train_decode, label="decoded function")
#     plt.legend(loc="best")

#     z_idx = np.searchsorted(train_input_rates[:, 0], 0) # input 0
#     rates_0 = spike_rates[z_idx] # spike rates at input 0
#     plt.figure()
#     plt.hist(decoders[rates_0>0], density=True, bins=40)

# plot_training_fit(tinfo.train_input_rates, tinfo.target_function, tinfo.spike_rates, tinfo.decoders.flatten())    